datetime
sortolas hogy latszodjon milyen idointervallum az adat +head,tail
utana hogy milyen weather a realis
filterezes: ahol min > max
ahol trip station name rossz
kitolteni a hianyzo zip code-ot station name alapjan
atteni stringrol int-re, hogy joinolni lehessen
outlierezes (boxplot)

In [ ]:
### majd ez alapjan kell joinilni
landmark_dict = {94107:'San Francisco',
                 94063:'Redwood City',
                 94301:'Palo Alto', 
                 94041:'Mountain View', 
                 95113:'San Jose'}
weather_df['landmark'] = weather_df['zip'].map(lambda x: landmark_dict[x])

### 1 - exercise
Load the datasets into pandas dataframes called trip, weather and station.<br><br>
In the trip dataframe create two new columns called 'start_datetime' and 'end_datetime'. These should contain the values from column 'Start Date' and 'End date' converted into datetime. <br><br>
In the weather dataframe create a new column called 'datetime' similarly from the 'Date' column.<br><br>
Display the datatypes of these columns to check the difference between the old and the new format.

In [1]:
### Your code here
import pandas as pd

weather = pd.read_csv('weather_filled.csv')
trip = pd.read_csv('trip_filled.csv')
station = pd.read_csv('station.csv')

trip['start_date'] = pd.to_datetime(trip['Start Date'], format='%m/%d/%Y %H:%M')
trip['end_date'] = pd.to_datetime(trip['End Date'], format='%m/%d/%Y %H:%M')
weather['datetime'] = pd.to_datetime(weather['Date'], format='%m/%d/%Y')

/home/esztersomos/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 1 - check yourself

In [11]:
if (trip['Start Date'] != trip['start_date'].dt.strftime('%-m/%-d/%Y %-H:%M')).sum() == 0:
    print('start_date column is successfully converted')
else:
    print('start_date column is NOT successfully converted')
if (trip['End Date'] != trip['end_date'].dt.strftime('%-m/%-d/%Y %-H:%M')).sum() == 0:
    print('end_date column is successfully converted')
else:
    print('end_date column is NOT successfully converted')
if (weather['Date'] != weather['datetime'].dt.strftime('%-m/%-d/%Y')).sum() == 0:
    print('datetime column is successfully converted')   
else:
    print('datetime column is NOT successfully converted')

start_date column is successfully converted
end_date column is successfully converted
datetime column is successfully converted
